In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import random
from tqdm import tqdm
print(tf.config.list_physical_devices('GPU'))

X_TRAIN_PATH = '../datasets/Dataset_UHCSDB/Patched/images_patched/'
Y_TRAIN_PATH = '../datasets/Dataset_UHCSDB/Patched/labels_png_patched/'

X_TRAIN_PATH_NEW = '../datasets/Dataset_KG/images/'
Y_TRAIN_PATH_NEW = '../datasets/Dataset_KG/masks/'

# Some images have an obstructing label on them so we ignore those
IGNORE_IMAGES = [
    '10Cs 11_patch_8.png', 'U-510 coeur_Champ_1_1_5_patch_24.png', 'V-294 peau_Champ_1_1_1_patch_24.png', 'v-448 b mr_Champ_1_1_4_patch_24.png', 'W-441_Champ_1_1_4_patch_24.png',
]

n_classes = 2

def load_and_augment_data(image_path, mask_path):
    images = []
    masks = []
    angles = [90, 180, 270]

    for file in os.listdir(image_path):
        if file.endswith('.png') and file not in IGNORE_IMAGES:
            img = cv2.imread(os.path.join(image_path, file), cv2.IMREAD_GRAYSCALE)
            images.append(img)
            mask = cv2.imread(os.path.join(mask_path, file), cv2.IMREAD_GRAYSCALE)
            masks.append(mask)

    augmented_images = []
    augmented_masks = []

    for img, mask in tqdm(zip(images, masks), total=len(images), desc="Augmenting data"):
        augmented_images.append(img)
        augmented_masks.append(mask)
        #for angle in angles:
        #    rotated_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE if angle == 90 else cv2.ROTATE_180 if angle == 180 else cv2.ROTATE_90_COUNTERCLOCKWISE)
        #    rotated_mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE if angle == 90 else cv2.ROTATE_180 if angle == 180 else cv2.ROTATE_90_COUNTERCLOCKWISE)
        #    augmented_images.append(rotated_img)
        #    augmented_masks.append(rotated_mask)

    return augmented_images, augmented_masks

#train_images, train_masks = load_and_augment_data(X_TRAIN_PATH, Y_TRAIN_PATH)
train_images, train_masks = load_and_augment_data(X_TRAIN_PATH_NEW, Y_TRAIN_PATH_NEW)

print("Number of training images:", len(train_images))
print("Shape of a single training image:", train_images[0].shape)
print("Number of training masks:", len(train_masks))
print("Shape of a single training mask:", train_masks[0].shape)

In [ ]:
# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(train_images, train_masks, test_size=0.2, random_state=42)

# Turn the lists into numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

print("Class values in Y_train:")
unique_classes = np.unique(Y_train)
print(unique_classes)   # [0 1 2 3]

# print shape of X_train and Y_train
print("Shape of X_train:", X_train.shape)  # (number_of_images, height, width, channels)
print("Shape of Y_train:", Y_train.shape)  # (number_of_images, height, width, channels)
# both are (76, 322, 322)

# Convert Y_train to categorical
from keras.utils import to_categorical

# for 2 classes
Y_train = (Y_train > 0).astype(np.uint8)
Y_test = (Y_test > 0).astype(np.uint8)

train_masks_cat = to_categorical(Y_train, num_classes=n_classes)
Y_train_cat = train_masks_cat.reshape((Y_train.shape[0], Y_train.shape[1], Y_train.shape[2], n_classes))

test_masks_cat = to_categorical(Y_test, num_classes=n_classes)
Y_test_cat = test_masks_cat.reshape((Y_test.shape[0], Y_test.shape[1], Y_test.shape[2], n_classes))

#from sklearn.utils import class_weight
#class_weights = class_weight.compute_class_weight(
#    class_weight='balanced',
#    classes=np.unique(Y_train),
#    y=Y_train.flatten()
#)
# Normalize class weights
#class_weights = class_weights / class_weights.sum()
#print("Class weights:", class_weights)  # [0.338 1.862 2.514 9.083]
class_weights = [0.82500247, 0.17499753]

In [ ]:
import u_net, u_net_512
model = u_net_512.unet_model_same_padding()
#model.compile(optimizer='adam', loss=[u_net.make_weighted_jaccard_loss(class_weights=class_weights)], metrics=[u_net.jaccard_coeff_multiclass])
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[u_net.jaccard_coeff_multiclass])
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer='adam', loss='categorical_focal_crossentropy', metrics=[u_net.jaccard_coeff_multiclass])

os.environ['SM_FRAMEWORK'] = 'tf.keras'
import segmentation_models as sm

LR = 0.0001
optimizer = tf.keras.optimizers.Adam(LR)
dice_loss = sm.losses.DiceLoss(class_weights=class_weights)
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + focal_loss
metrics = [
    sm.metrics.IOUScore(threshold=0.5, class_weights=class_weights),
    sm.metrics.FScore(threshold=0.5, class_weights=class_weights)
]

model.compile(optimizer=optimizer, loss=total_loss, metrics=metrics)

# Callbacks
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True)

history = model.fit(
    X_train,
    Y_train_cat,
    batch_size=4,
    verbose=1,
    epochs=50,
    validation_data=(X_test, Y_test_cat),
    shuffle=True,
    callbacks=[
        early_stopping,
        model_checkpoint
        ],
)

In [ ]:
model = tf.keras.models.load_model('best models/best_small_cross_entropy.keras')

In [ ]:
model = tf.keras.models.load_model('best models/best_large_focal_loss.keras', custom_objects={
    'jaccard_coeff_multiclass': u_net.jaccard_coeff_multiclass
})

In [ ]:
os.environ['SM_FRAMEWORK'] = 'tf.keras'
import segmentation_models as sm

model = tf.keras.models.load_model('best models/best_large_mixed_loss.keras', compile=False)

LR = 0.0001
optimizer = tf.keras.optimizers.Adam(LR)
dice_loss = sm.losses.DiceLoss(class_weights=class_weights)
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + focal_loss
metrics = [
    sm.metrics.IOUScore(threshold=0.5, class_weights=class_weights),
    sm.metrics.FScore(threshold=0.5, class_weights=class_weights)
]

model.compile(optimizer=optimizer, loss=total_loss, metrics=metrics)

In [ ]:
# Evaluate the model
# Evaluate the model in smaller batches to avoid running out of memory
loss, accuracy = model.evaluate(X_test, Y_test_cat, batch_size=4, verbose=1)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

In [ ]:
#IOU FOR OLD DATASET
y_pred = model.predict(X_test, batch_size=4)
y_pred_argmax=np.argmax(y_pred, axis=3)

#Using built in keras function
from keras.metrics import MeanIoU
n_classes = 4
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(Y_test, y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())


#To calculate I0U for each class...
values = IOU_keras.total_cm.numpy()
print(values)
class1_IoU = values[0,0]/(values[0,0] + values[0,1] + values[0,2] + values[0,3] + values[1,0]+ values[2,0]+ values[3,0])
class2_IoU = values[1,1]/(values[1,1] + values[1,0] + values[1,2] + values[1,3] + values[0,1]+ values[2,1]+ values[3,1])
class3_IoU = values[2,2]/(values[2,2] + values[2,0] + values[2,1] + values[2,3] + values[0,2]+ values[1,2]+ values[3,2])
class4_IoU = values[3,3]/(values[3,3] + values[3,0] + values[3,1] + values[3,2] + values[0,3]+ values[1,3]+ values[2,3])

print("IoU for class1 is: ", class1_IoU)
print("IoU for class2 is: ", class2_IoU)
print("IoU for class3 is: ", class3_IoU)
print("IoU for class4 is: ", class4_IoU)

In [ ]:
from keras.metrics import MeanIoU

# IOU FOR NEW DATASET
y_pred_new = model.predict(X_test, batch_size=4)
y_pred_argmax_new = np.argmax(y_pred_new, axis=3)

# Using built-in Keras function to calculate Mean IoU
IOU_keras_new = MeanIoU(num_classes=n_classes)
IOU_keras_new.update_state(Y_test, y_pred_argmax_new)
print("Mean IoU for new dataset =", IOU_keras_new.result().numpy())

# To calculate IoU for each class
values_new = IOU_keras_new.total_cm.numpy()
print(values_new)
class1_IoU_new = values_new[0, 0] / (values_new[0, 0] + values_new[0, 1] + values_new[1, 0])
class2_IoU_new = values_new[1, 1] / (values_new[1, 1] + values_new[1, 0] + values_new[0, 1])

print("IoU for class1 (new dataset):", class1_IoU_new)
print("IoU for class2 (new dataset):", class2_IoU_new)

In [ ]:

# plot the training history
import matplotlib.pyplot as plt
def plot_history(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['iou_score'])
    plt.plot(history.history['val_iou_score'])
    plt.title('Model IoU')
    plt.ylabel('IoU')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

plot_history(history)


In [ ]:
#random.seed(42)
# Pick a random test image
import matplotlib.pyplot as plt
idx = random.randint(0, X_test.shape[0] - 1)
test_img = X_test[idx]
true_mask = Y_test[idx]

# Predict the mask
pred_mask = model.predict(np.expand_dims(test_img, axis=0))
pred_mask = np.argmax(pred_mask, axis=-1)[0]

# Plot the images
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.title("Test Image")
plt.imshow(test_img, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title("True Mask")
plt.imshow(true_mask, cmap='nipy_spectral', vmin=0, vmax=n_classes-1)
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title("Predicted Mask")
plt.imshow(pred_mask, cmap='nipy_spectral', vmin=0, vmax=n_classes-1)
plt.axis('off')

plt.show()
